In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [11]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=1e-4, model=model, c1=1e-4, tau=0.1, line_search_method="const")
opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.43445923924446106
epoch:  1, loss: 0.2384142130613327
epoch:  2, loss: 0.13895511627197266
epoch:  3, loss: 0.08712276816368103
epoch:  4, loss: 0.060338135808706284
epoch:  5, loss: 0.04675840958952904
epoch:  6, loss: 0.04001617804169655
epoch:  7, loss: 0.03673045337200165
epoch:  8, loss: 0.03515221178531647
epoch:  9, loss: 0.034401919692754745
epoch:  10, loss: 0.03404721990227699
epoch:  11, loss: 0.03387952968478203
epoch:  12, loss: 0.0337994359433651
epoch:  13, loss: 0.033760275691747665
epoch:  14, loss: 0.03374011814594269
epoch:  15, loss: 0.03372875228524208
epoch:  16, loss: 0.03371612727642059
epoch:  17, loss: 0.03369523212313652
epoch:  18, loss: 0.03368362784385681
epoch:  19, loss: 0.03367273882031441
epoch:  20, loss: 0.033651143312454224
epoch:  21, loss: 0.03363918140530586
epoch:  22, loss: 0.0336291529238224
epoch:  23, loss: 0.033606477081775665
epoch:  24, loss: 0.033594027161598206
epoch:  25, loss: 0.033584319055080414
epoch:  26, loss: 

In [12]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.7557700806203207
Test metrics:  R2 = 0.702852068839014
